# DPCRO_SL Guide

In [5]:
from CRO_SL import *
from AbsObjetiveFunc import *
from SubstrateReal import *

The first step will be to define the objetive function that will be optimized, this will be done taking as a base the class AbsObjetiveFunc.
In this case, we will optimize a very simple function, simply adding the squared components of the vector. 

In [3]:

"""
All the following methods will have to be implemented for the algorithm to work properly
with the same inputs, except for the constructor 
"""
class SumSquare(AbsObjetiveFunc):
    """
    This is the constructor of the class, here is where the objetive function can be setted up.
    In this case we will only add the size of the vector as a parameter.
    """
    def __init__(self, size):
        self.size = size
        self.opt = "min" # it can be "max" or "min"

        # we call the constructor of the superclass with the size of the vector
        # and wether we want to maximize or minimize the function 
        super().__init__(self.size, self.opt)
    
    """
    This will be the objetive function, that will recieve a vector and output a number
    """
    def objetive(self, solution):
        return (solution**2).sum()
    
    """
    This will be the function used to generate random vectors
    """
    def random_solution(self):
        return 200*np.random.random(self.size)-100
    
    """
    This will be the function that will repair solutions, or in other words, makes a solution
    outside the domain of the function into a valid one.
    If this is not needed simply return "solution"
    """
    def check_bounds(self, solution):
        return np.clip(solution, -100, 100)

In [7]:
objfunc = SumSquare(30)

Now that we have our objetive function, we can proceed to initialize the optimization algorithm.

First, we configure the hyperparameters of the algorithm in a dictionary, a detailed list of the parameters needed can be found on the README.md file. Here we use a configuration that should work ok in most cases.

In [ ]:
params = {
    "popSize": 100,
    "rho": 0.6,
    "Fb": 0.98,
    "Fd": 0.2,
    "Pd": 0.8,
    "k": 3,
    "K": 20,
    "group_subs": True,

    "stop_cond": "ngen",
    "time_limit": 4000.0,
    "Ngen": 200,
    "Neval": 10e5,
    "fit_target": 1000,

    "verbose": True,
    "v_timer": 1,

    "dynamic": True,
    "dyn_method": "success",
    "dyn_metric": "avg",
    "dyn_steps": 10,
    "prob_amp": 0.01
}

Since this algorithm relies on the mixing of different mutation and recombination operators, we will create the list of operators to be used. To create an operator we create an instance of the class SubstrateReal in case we are working with real valued vectors, and SubstrateInt if we are working with discrete valued vectors.

Each operator has it's own parameters that will be specified in the creation of the operator

In [6]:
operators = [
    SubstrateReal("DE/rand/1", {"F": 0.7, "Pr":0.8}),
    SubstrateReal("DE/best/2", {"F": 0.7, "Pr":0.8}),
    SubstrateReal("DE/current-to-best/1", {"F": 0.7, "Pr":0.8}),
    SubstrateReal("DE/current-to-rand/1", {"F": 0.7, "Pr":0.8})
]

Now we simply an instance of the CRO_SL algorithm with the previous data. 

In [ ]:
cro_alg = CRO_SL(objfunc, operators, params)